In [88]:
import os
from pprint import pprint

from msl.equipment import EquipmentRecord, ConnectionRecord, Backend

from msl.equipment.resources.thorlabs import MotionControl
# ensure that the Kinesis folder is available on PATH
os.environ['PATH'] += os.pathsep + 'C:/Program Files/Thorlabs/Kinesis'



#FUNCIONES QUE QUIERO
#1) DECIME QUE STEP MOTORS/DRIVERS HAY CONECTADOS A LA COMPU Y DAME LOS SERIAL NUMBER'S
#2) CONECTATE A LOS MOTORES Y LLEVALOS AL HOME XQ ES OBLIGATORIO PARA PODER MOVERLOS
#3) WAIT QUE YA VENÌA..
#4) MOVELOS A UNA CIERTA POSICIÒN X,Y
#5) DESCONECTATE DE LOS MOTORES

class ThorlabsStageWithStepMotors:

    def what_is_connected(self):
        print('Building the device list...')
        MotionControl.build_device_list()
        n_devices = MotionControl.get_device_list_size()
        if n_devices == 0:
            print('There are no devices in the device list')
            sys.exit(0)
        elif n_devices == 1:
            print('There is 1 device in the device list')
        else:
            print('There are {} devices in the device list'.format(n_devices))
        all_devices = MotionControl.get_device_list()
        print('The serial numbers of all the devices are: {}'.format(all_devices))
    
        
        
        
    def set_x_and_y_motor(self,serialx,serialy):        
        recordx = EquipmentRecord(
            manufacturer='Thorlabs',
            model='KST101',
            serial= str(serialx),  # update the serial number for your KST101
            connection=ConnectionRecord(
                backend=Backend.MSL,
                address='SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll',
            ),
        )
        recordy = EquipmentRecord(
            manufacturer='Thorlabs',
            model='KST101',
            serial= str(serialy),  # update the serial number for your KST101
            connection=ConnectionRecord(
                backend=Backend.MSL,
                address='SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll',
            ),
        )
        print('The motor in the x direction is the one with serial number # ' + str(serialx))
        print('The motor in the y direction is the one with serial number # ' + str(serialy))
        motorx = recordx.connect()
        motory = recordy.connect()
        print('Connected to {}'.format(motorx))
        print('Connected to {}'.format(motory))
        # load the configuration settings (so that we can use the get_real_value_from_device_unit() method)
        motorx.load_settings()
        motory.load_settings()
        # start polling at 200 ms
        motorx.start_polling(200)
        motory.start_polling(200)
        print('Homing both motors...')
        motorx.home()
        motory.home()
        self.wait(0,motorx)
        self.wait(0,motory)
        print('Homing done. X motor is at position {} [mm]'.format(motorx.get_position()) + ' and Y motor at position {} [mm]'.format(motorx.get_position()))
        
        
        
        
        
        
    def wait(self,value,motor):
        motor.clear_message_queue()
        message_type, message_id, _ = motor.wait_for_message()
        while message_type != 2 or message_id != value:
            position = motor.get_position()
            real = motor.get_real_value_from_device_unit(position, 'DISTANCE')
            print('  at position {} [device units] {} [real-world units]'.format(position, real))
            message_type, message_id, _ = motor.wait_for_message()
            
    def move_to_x_y_position(self,x,y):
        if motorx.get_position() == 0 and motory.getposition() == 0:
            motorx.move_to_position(motorx.get_device_unit_from_real_value(x,0))
            motory.move_to_position(motory.get_device_unit_from_real_value(y,0))
            print('Moving done. At position {} [device units]'.format(motorx.get_position()))
            print('Moving done. At position {} [device units]'.format(motory.get_position()))
            
    def disconnect_stage(self):
        # stop polling and close the connection
        motorx.stop_polling()
        motory.stop_polling()
        motory.disconnect()
        motorx.disconnect()
        
            

        

In [85]:
stage = ThorlabsStageWithStepMotors()

In [86]:
stage.what_is_connected()

Building the device list...
There are 2 devices in the device list
The serial numbers of all the devices are: ['26001683', '26001718']


In [90]:
stage.set_x_and_y_motor(26001683,26001718)

The motor in the x direction is the one with serial number # 26001683
The motor in the y direction is the one with serial number # 26001718


KCubeStepperMotor<Thorlabs|KST101|26001718 at SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll> The function did not return True.
KCubeStepperMotor.SCC_LoadSettings(b'26001718',) -> False


Connected to KCubeStepperMotor<Thorlabs|KST101|26001683>
Connected to KCubeStepperMotor<Thorlabs|KST101|26001718>


ThorlabsError: KCubeStepperMotor<Thorlabs|KST101|26001718 at SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll>
The function did not return True.
KCubeStepperMotor.SCC_LoadSettings(b'26001718',) -> False

In [91]:
stage.move_to_x_y_position(1.0,1.0)

NameError: name 'motorx' is not defined

In [89]:
stage.disconnect_stage()

NameError: name 'motorx' is not defined

In [74]:
stage.set_x_and_y_motor(26001683,26001749)

The motor in the x direction is the one with serial number # 26001683
The motor in the y direction is the one with serial number # 26001749
Connected to KCubeStepperMotor<Thorlabs|KST101|26001683>
Connected to KCubeStepperMotor<Thorlabs|KST101|26001749>
Homing both motors...
  at position -71828 [device units] -0.0357598 [real-world units]
  at position -71828 [device units] -0.0357598 [real-world units]
  at position -262826 [device units] -0.1308488 [real-world units]
  at position -280555 [device units] -0.1396752 [real-world units]
  at position -280555 [device units] -0.1396752 [real-world units]
  at position -240167 [device units] -0.1195679 [real-world units]
  at position 17 [device units] 8.5e-06 [real-world units]
  at position 17 [device units] 8.5e-06 [real-world units]
  at position 21254 [device units] 0.0105814 [real-world units]
  at position 61672 [device units] 0.0307036 [real-world units]
  at position 102083 [device units] 0.0508224 [real-world units]
  at position

KCubeStepperMotor<Thorlabs|KST101|26001749 at SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll> FT_GenericFunctionFail: The function failed to complete succesfully


  at position 182820 [device units] 0.0910175 [real-world units]


ThorlabsError: KCubeStepperMotor<Thorlabs|KST101|26001749 at SDK::Thorlabs.MotionControl.KCube.StepperMotor.dll>
FT_GenericFunctionFail: The function failed to complete succesfully